<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-13.-Operator-Overloading:-Doing-It-Right" data-toc-modified-id="Chapter-13.-Operator-Overloading:-Doing-It-Right-1">Chapter 13. Operator Overloading: Doing It Right</a></span><ul class="toc-item"><li><span><a href="#Overloading-unary-operators-on-Vector" data-toc-modified-id="Overloading-unary-operators-on-Vector-1.1">Overloading unary operators on Vector</a></span></li><li><span><a href="#Overloading-infix-operators-on-Vector" data-toc-modified-id="Overloading-infix-operators-on-Vector-1.2">Overloading infix operators on Vector</a></span></li></ul></li></ul></div>

# Chapter 13. Operator Overloading: Doing It Right

In [459]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):   
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):   
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())   
            outer_fmt = '<{}>'   
        else:
            coords = self
            outer_fmt = '({})'   
        components = (format(c, fmt_spec) for c in coords)   
        return outer_fmt.format(', '.join(components))   

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

## Overloading unary operators on Vector

In [535]:
def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

def __neg__(self):
    return Vector(-x for x in self)   

def __pos__(self):
    return Vector(self)   

Vector.__abs__ = __abs__
Vector.__neg__ = __neg__
Vector.__pos__ = __pos__

In [538]:
v = Vector([1,1])
abs(v)

1.4142135623730951

In [539]:
-v

Vector([-1.0, -1.0])

In [540]:
+v

Vector([1.0, 1.0])

## Overloading infix operators on Vector

In [460]:
# itemwise add to Vectors of unequal length
import itertools
def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)  
        return Vector(a + b for a, b in pairs)  
Vector.__add__ = __add__

In [461]:
v1 = Vector([3, 4, 5, 6])
v3 = Vector([1, 2])
v1 + v3

Vector([4.0, 6.0, 5.0, 6.0])

In [462]:
# supports non-Vector operands as well
v1 + (10,20,30)

Vector([13.0, 24.0, 35.0, 6.0])

In [463]:
v2d = Vector2d(10, 20)
v1 + v2d

Vector([13.0, 24.0, 5.0, 6.0])

operator order is important though

In [464]:
v2d + v1

TypeError: unsupported operand type(s) for +: 'Vector2d' and 'Vector'

To make the mixed-type additions work in any order, we need to implement the `Vector.__radd__` method

In [465]:
def __radd__(self, other):  # 
        return self + other
Vector.__radd__ = __radd__

In [466]:
v2d + v1

Vector([13.0, 24.0, 5.0, 6.0])

Use goose typing to overload * for scalar multiplication of Vector

In [469]:
def __mul__(self, scalar):
    if isinstance(scalar, numbers.Real):  # 
        return Vector(n * scalar for n in self)
    else:  # 
        return NotImplemented

def __rmul__(self, scalar):
    return self * scalar  # 

Vector.__mul__ = __mul__
Vector.__rmul__ = __rmul__

In [470]:
14 * v1

Vector([42.0, 56.0, 70.0, 84.0])

In [471]:
v1 * True

Vector([3.0, 4.0, 5.0, 6.0])

In [472]:
from fractions import Fraction
v1 * Fraction(1,3)

Vector([1.0, 1.3333333333333333, 1.6666666666666665, 2.0])

Python > 3.5 supports matrix multiplication through the @ operator

In [473]:
def __matmul__(self, other):
    try:
        return sum(a * b for a, b in zip(self, other))
    except TypeError:
        return NotImplemented

def __rmatmul__(self, other):
    return self @ other

Vector.__matmul__ = __matmul__
Vector.__rmatmul__ = __rmatmul__

In [474]:

va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
va @ vz

38.0